## 총점 및 순위 계산

### 1. 읽어들일 파일명 지정 ==> 가지고 있는 파일명으로 수정필요

In [2]:
########### file_name 선언 ##################################
전처리데이터셋 = '남양주_전처리데이터셋.csv'
##########################################################

### 2. 화면에 출력되는 칼럼명을 확인한 후, 필요한 칼럼명을 복사하여 붙여넣기 (수정 불필요)

In [14]:
import pandas as pd
df = pd.read_csv(f'{전처리데이터셋}', header=0, encoding='cp949', engine='python')
df.set_index('gid', inplace=True)
전체칼럼 = df.columns.tolist()
print ("여기 칼럼명을 확인하신 후 점수계산에 필요한 칼럼명만 복사 하세요.\n", 전체칼럼)

여기 칼럼명을 확인하신 후 점수계산에 필요한 칼럼명만 복사 하세요.
 ['건물1', '건물2', '건물3', '관공서', '전기차', '주차장', '충전소', '변압기', '전기안전관', 'MAPLINK_ID', '교통량', '경제인구', 'lon', 'lat']


### 3. 하단에 있는 "점수칼럼"에 점수로 환산할 대상인 칼럼을 수정하도록 함 ==> 수정 필요

In [1]:
#점수칼럼 =  ['건물1', '건물2', '건물3', '관공서', '도서관', '경제인구', '교통량', '전기차', '관광지']
점수칼럼 =  ['건물1', '건물2', '건물3', '관공서', '전기차', '주차장', '충전소', '변압기', '전기안전관', '교통량', '경제인구']
print("입력한 점수컬럼명은 다음과 같음\n", 점수칼럼)
print(len(점수칼럼))
가중치 = [1] * len(점수칼럼)

입력한 점수컬럼명은 다음과 같음
 ['건물1', '건물2', '건물3', '관공서', '전기차', '주차장', '충전소', '변압기', '전기안전관', '교통량', '경제인구']
11


### 4. 건물 가중치 선언 ==> 필요시 수정
- 각 건물과 관광지에 대한 가중치를 수기로 선언함
- 필요시 가중치 조절 하십시오 
- 필요시 가중치 추가 또는 삭제 하십시오

In [23]:
#점수칼럼  순서대로 점수칼럼 갯수만큼 가중치를 입력하십시오

#예를들어 이렇게 가중치를 주고 싶을 경우에는
#건물1가중치=1
#건물2가중치 = 2
#건물3가중치  = 3
#관광지가중치=1
#도서관가충치=1
#경제인구가중치=1
#전기차가중치=1
#아래 처럼 입력합니다.

##### 여기만 수정하면 됩니다 ###############################
가중치 = [1,2,3,1,1,1,1,1,1,1,1]      # 점수칼럼 갯수 만큼, 구분은 쉼표
##### 여기만 수정하면 됩니다 ###############################


# 수정 불필요
if (len(가중치) != len(점수칼럼)):
    print("입력한 가중치 갯수가 점수칼럼 갯수와 다르므로 확인후 다시 입력 바랍니다")
else:
    print("다음 줄을 실행 하세요")

다음 줄을 실행 하세요


### 5. 이하 별도 언급 시까지 수정 불필요
- 6.1에서 수정 필요

In [24]:
df = pd.read_csv(f'{전처리데이터셋}', header=0, encoding='cp949', engine='python')
df.set_index('gid', inplace=True)

################## QGIS에서 작업한 전처리 결과 CSV 읽기 ####################################################
pre_process_set = pd.read_csv(f'{전처리데이터셋}', header=0, encoding='cp949', engine='python')
pre_process_set = pre_process_set.sort_values(by='gid')

# Null 이 있을 수 있는 칼럼은 Null을 0로 변경
pre_process_set.fillna(0, inplace=True)
##################  QGIS에서 작업한 전처리 결과 CSV 읽기 끝###################################################

################## MinMax정규화 ##########################################################################
from sklearn.preprocessing import MinMaxScaler
scaler = MinMaxScaler()

# 점수칼럼 정규화
norm = pre_process_set.copy()
norm.fillna(0, inplace=True)
norm.set_index('gid', inplace=True)

norm[점수칼럼] = scaler.fit_transform(norm.loc[:,점수칼럼])
norm.reset_index(inplace=True)
################## 정규화 끝################################################################################


# 총점 계산
df = norm.copy()

for i in range(0, len(점수칼럼)):
    name = 점수칼럼[i]
    df[name] = df[name] * 가중치[i] 

df.set_index(['gid','주차장','lon','lat'], inplace=True)

df['sum'] = df.sum(1)
df.reset_index(inplace=True)

# 등수 계산 
df['순위'] = df['sum'].rank(method='dense', ascending=False).astype(int)

final = df.sort_values(by='순위')
final.reset_index(drop=True, inplace=True)

# 상위 20개만 학인
display("결과", final.head(20))

# 최종 결과 CSV 파일쓰기
final.to_csv('결과_총점.csv', index=False, sep=',', encoding='cp949')

'결과'

,gid,주차장,lon,lat,건물1,건물2,건물3,관공서,전기차,충전소,변압기,전기안전관,MAPLINK_ID,교통량,경제인구,sum,순위
0,다사74ba54aa,1.0,127.212590,37.586169,1.000000,0.185185,0.0,0.0,0.50,0.0,0.0,0.000000,76,0.092593,0.289523,78.067300,1
1,다사72ba64ba,0.0,127.189544,37.680751,0.881579,0.000000,0.0,0.0,0.00,0.0,0.0,0.000000,67,0.000000,0.013639,67.895218,2
2,다사76aa71aa,0.0,127.229023,37.739435,0.868421,0.000000,0.0,0.0,0.00,0.0,0.0,0.000000,66,0.000000,0.002790,66.871211,3
3,다사75bb71aa,0.0,127.226186,37.739429,0.855263,0.000000,0.0,0.0,0.00,0.0,0.0,0.000000,65,0.000000,0.006820,65.862083,4
4,다사72aa68bb,0.5,127.183711,37.719043,0.842105,0.074074,0.0,0.5,0.00,0.0,0.0,0.000000,64,0.037037,0.129262,65.582479,5
5,다사80bb61ab,0.0,127.283190,37.651667,0.789474,0.000000,0.0,0.0,0.00,0.0,0.0,0.000000,60,0.000000,0.024179,60.813652,6
6,다사74ba68ab,0.0,127.212095,37.714609,0.789474,0.000000,0.0,0.0,0.00,0.0,0.0,0.000000,60,0.000000,0.013949,60.803423,7
7,다사74aa63ab,0.0,127.206599,37.669528,0.776316,0.000000,0.0,0.0,0.00,0.0,0.0,0.000000,59,0.000000,0.046187,59.822503,8
8,다사71ba61bb,0.0,127.178310,37.655934,0.736842,0.037037,0.0,1.0,0.00,0.0,0.0,0.000000,56,0.018519,0.093304,57.885702,9
9,다사75ba71aa,0.0,127.223348,37.739422,0.750000,0.000000,0.0,0.0,0.00,0.0,0.0,0.000000,57,0.000000,0.008060,57.758060,10


## 6. 지도로 보기
- 인터넷 환경이 안되면 생략

### 6.1 지도로 보기 선택 사항
- 상단에서 도출된 결과값들 중에서 상위 3개만 지도로 보여주기
- Google Map 처럼 보여지며, 마우스로 스크롤하면 지도 크기가 Zoom 됨
- 이를 위해서 Anaconda Promprt 창을 열고 아래 함수를 복사하여 그대로 엔터
 - pip install folium

#### 수정 필요 - 수정 안하면 상위 20개를 조회

In [25]:
# 보고 싶은 상위권 갯수를 입력하세요
지도보기갯수 = 20   # 보고싶은 상위권 갯수를 여기에 입력, 만약 3이면 최상위 3개만 보겠다는 의미

### 6.2 수정 불필요

In [26]:
# 지도로 보여주기
import folium
print(f"보고자 하는 상위권 갯수는 {지도보기갯수}개 입니다.")
for i in range(지도보기갯수):
    지도보기 = folium.Map(location=[final.loc[i,'lat'],final.loc[i,'lon']], zoom_start = 15)
    folium.Marker([final.loc[i,'lat'],final.loc[i,'lon']]).add_to(지도보기)
    print (final.loc[i,'lat'],final.loc[i,'lon'])
    j = i+1
    display(f'상위 {j}번째 격자의 센터 위치 보기', 지도보기)

보고자 하는 상위권 갯수는 20개 입니다.
37.58616874 127.2125899


'상위 1번째 격자의 센터 위치 보기'

37.68075137 127.1895437


'상위 2번째 격자의 센터 위치 보기'

37.73943527 127.2290231


...

'상위 18번째 격자의 센터 위치 보기'

37.71009501 127.2092756


'상위 19번째 격자의 센터 위치 보기'

37.70102366 127.1866232


'상위 20번째 격자의 센터 위치 보기'

### 6. Box Plot 보기 선택 사항 - 수정 불필요
- 상기 선택사항 실행여부와 무관하게 9번은 실행가능함
- 값들이 몇점대에 몰려 있는지 확인해보는 Box Plot 그래프
- 맨 하단부에 몰려 있을 가능성이 높음. 이유는 절반이상이 0점이기 때문

In [ ]:
#참고) 총점 분포도
import matplotlib.pyplot as plt

In [ ]:
plt.figure()
plt.boxplot(final['sum'])
plt.show()

끝